In [92]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [131]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=2, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [132]:
def calcBc():
    r = 1/Integrate(1,mesh)*Integrate(x,mesh)
    return r
calcBc()

-5.080028269950915e-17

In [94]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [134]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)
vol = Parameter(1)
vol.Set(Integrate(surf_t,mesh))
vol_inv = Parameter(1)
vol_inv.Set(1/Integrate(surf_t,mesh))
# bc prep
bc_tx = CoefficientFunction(x)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)

# bc constraints
beta0 = 1e-3
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_int = Parameter(1)
bc_int.Set(Integrate(bc_tx,mesh))
bc_x.Set(calcBc())

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [135]:
dJOmega.Assemble()
print("curvol: ", vol.Get())
print("Initial bc: ", bc_x.Get())
print(dJOmega.vec[0:5])

curvol:  12.566430839284152
Initial bc:  -5.080028269950915e-17
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [136]:
mesh.UnsetDeformation()
test = CoefficientFunction((0.8,1.2))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [99]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print("curvol: ", Integrate(1,mesh))
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
print(dJOmega.vec)

curvol:  12.56643083928415
0.7999999999999996
0.7999999999999996
  1.89735e-19
 -0.000392056

  0.000392056
 1.35525e-19

  -4.40457e-20
 0.000392056

  -0.000392056
 -8.80914e-20

  0.000123018
 -0.0003812

  0.000235182
 -0.000324147

  0.000324147
 -0.000235182

  0.0003812
 -0.000123018

  0.0003812
 0.000123018

  0.000324147
 0.000235182

  0.000235182
 0.000324147

  0.000123018
 0.0003812

  -0.000123018
 0.0003812

  -0.000235182
 0.000324147

  -0.000324147
 0.000235182

  -0.0003812
 0.000123018

  -0.0003812
 -0.000123018

  -0.000324147
 -0.000235182

  -0.000235182
 -0.000324147

  -0.000123018
 -0.0003812

  6.77626e-21
 -1.0842e-19

  -8.13152e-20
      0

  -1.0842e-19
 -2.71051e-20

  6.94567e-20
 1.0842e-19

       0
 -1.35525e-19

  5.42101e-20
 -2.71051e-20

  -1.35525e-20
 1.0842e-19

  -5.42101e-20
 1.35525e-20

  5.42101e-20
 -8.13152e-20

  2.71051e-20
 -2.71051e-20

  1.0842e-19
 -1.81053e-20

       0
 5.42101e-20

  2.71051e-20
 1.0842e-19

  -5.42101e-20
 -

## try to do this as shape optimization

$\Large \frac{\partial }{\partial x} \beta(bc(x)-bc_0)^2 =$ <br>
$\Large 2(\frac{1}{bc(x)}-bc_0)\cdot \frac{\partial }{\partial x}(\frac{1}{bc(x)})$

In [122]:
# don't use 2 deformations, just set bc_0x to something else, let's see if we get there:
mesh.UnsetDeformation()
bc_0x = 3

gfset = GridFunction(VEC)

dJOmega2 = LinearForm(VEC)
#dJOmega2 += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)
#dJOmega2 += ((2*beta*((1/bc_x-bc_0x))*(1/bc_x)*dx)).DiffShape(X)
dJOmega2 += (2*(1/(vol_inv*bc_int)-bc_0x)*(1/(vol_inv*bc_int))*dx).DiffShape(X)

gfset = GridFunction(VEC)

b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega2.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [125]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
bc_x.Get()

-5.0800282699509165e-17

In [124]:
import time
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 100

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    #bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh)) # needed for multiple iterations (already done once before)
    vol_inv.Set(1/Integrate(surf_t,mesh))
    bc_int.Set(Integrate(bc_tx,mesh))
    print(1/Integrate(1,mesh)*Integrate(x,mesh))
    #print(bc_x.Get())
    dJOmega2.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.1 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    
    time.sleep(0.1)

-5.080028269950915e-17
1.429376008822665e-05
0.0037971088310493222
0.0038118208830393687
0.0038260633696637607
0.0038399070568818842
0.003853360777934556
0.0038664331665739476
0.0038791326557327597
0.003891467476622981
0.0039034456582471176
0.003915075027301849
0.003926363208455309
0.003937317624979514
0.003947945499715264
0.003958253856350968
0.00396824952099261
0.003977939124005045
0.003987329102100785
0.003996425700657394
0.004005234976238724
0.004013762799300506
0.004022014857057539
0.004029996656491026
0.004037713527474182
0.004045170625995862
0.004052372937459982
0.004059325280040356
0.0040660323080704995
0.004072498515448639
0.0040787282390354045
0.00408472566202808
0.004090494817288327
0.004096039590606752
0.004101363723882562
0.0041064708182014815
0.004111364336791161
0.004116047607836409
0.004120523827133511
0.004124796060566072
0.004128867246381856
0.004132740197250145
0.00413641760208111
0.004139902027584064
0.004143195919543373
0.004146301603790125
0.004149221286844041
0.0

KeyboardInterrupt: 

In [ ]:
bc_x.Get()